```{currentmodule} optimap
```

In [ ]:
from optimap.utils import jupyter_render_animation as render

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/activation.ipynb>`, or a {download}`python script <converted/activation.py>` with code cells. We highly recommend using [Visual Studio Code](#vscode) to execute this tutorial.
```

# Tutorial 6: Activation Maps

This tutorial demonstrates how to compute and visualize activation maps from cardiac optical mapping data using ``optimap``. Activation maps display local activation times (LATs), which indicate when the tissue becomes electrically activated. These maps are crucial for understanding cardiac electrical activity and identifying abnormalities.

Computing local activation times corresponds to determining when the optical signal in a given pixel passes a certain pre-defined threshold or intensity value. For instance, if the optical trace is normalized and fluctuates between [0, 1] then the tissue could be defined as being 'electrically activated' when the time-series rises above or below 0.5 (depending on the fluorescent indicator and polarity of the signal).

Here, we will use an example data from {cite:t}`Rybashlykov2022` in which a planar action potential wave propagates across the ventricle of a mouse heart.

In [ ]:
import optimap as om
import monochrome as mc
import numpy as np
import matplotlib.pyplot as plt

filename = om.download_example_data("doi:10.5281/zenodo.5557829/mouse_41_120ms_control_iDS.mat")
video = om.load_video(filename)
metadata = om.load_metadata(filename)
print(f"Loaded video with shape {video.shape} and metadata {metadata}")
frequency = metadata["frequency"]

The `mouse_41_120ms_control_iDS.mat` file from the [Zenodo dataset](https://doi.org/10.5281/zenodo.5557829) shows a induced pacing beats in a mouse heart. The {func}`load_metadata` function loads the metadata from the MATLAB file, in this case the acquisition frame rate. We visualize the video using [Monochrome](https://github.com/sitic/Monochrome):

In [ ]:
# Show video
mc.show(video, name=filename.name, metadata=metadata)

In [ ]:
from IPython.display import Video
Video('https://cardiacvision.ucsf.edu/sites/g/files/tkssra6821/f/optimap-mouse_41_120ms_control_iDS_monochrome.mp4', html_attributes='controls autoplay loop width="100%"')

We use ``optimap``'s {func}`background_mask` function to create a mask of the heart:

In [ ]:
# remove background by masking
mask = om.background_mask(video[0], show=False)
mask = om.image.dilate_mask(mask, iterations=5, show=False)
om.image.show_mask(mask, video[0], title="Background mask");

Computing activation maps is highly noise-sensitive, so we need to need to apply strong filtering to the data. Here we use a spatio-temporal Gaussian filter and a spatial mean filter to smooth the data.

In [ ]:
video_filtered = om.video.smooth_spatiotemporal(video, sigma_temporal=1, sigma_spatial=1)
video_filtered = om.video.mean_filter(video_filtered, size_spatial=5)

# Normalize the video using a pixelwise sliding window
video_norm = om.video.normalize_pixelwise_slidingwindow(video_filtered, window_size=200)
video_norm[:, mask] = np.nan

Here we used a pixelwise sliding window normalization to normalize the traces of each pixel to the range [0, 1]. We could have also used a regular pixel-wise normalization or a normalization off a frame-wise difference filtered video, see [Tutorial 2](signal_extraction.ipynb).

Because the mouse heart was stained with the voltage-sensitive dye Di-4-ANEPPS, the tissue becomes darker when it depolarizes (negative signal / polarity):

In [ ]:
om.show_video_pair(video, video_norm, title1="original video",
                   title2="normalized video", interval=10)

# Or in Monochrome:
#
# mc.show(video, name="original video")
# mc.show(video_norm, name="normalized video")

In [ ]:
render(lambda: om.show_video_pair(video, video_norm, title1="original video", title2="normalized video", interval=20))

We can use the {func}`activation.find_activations` function to find pacing beats. Because the activation wave is so fast, we can just average the signal over the whole heart to identify when the pacing beat occurs:

In [ ]:
activations = om.activation.find_activations(1 - video_norm, fps=frequency)
print(f"Found {len(activations)} activation events at frames: {activations}")

The function return a list of activation times in frames and plots detected activations with a red line. We plotted `1 - signal` here to show the normal action potential shape.

To get a more accurate estimate when the first activation occurs per pacing beat we can manually select a pixel close to the pacing site and run it again:

In [ ]:
# Select a pixel position close to the pacing site to find activation events
#
# traces, coords = om.select_traces(video_norm[:500], size=10, ref_frame=video[0])
# trace = om.extract_traces(video_norm, coords[0], size=10)
# activations = om.activation.find_activations(1 - trace)

# Here hardcoded position (141, 100) for demo purposes
fig, axs = plt.subplot_mosaic('ABB', figsize=(10, 2))
om.show_positions([(141, 100)], video[0], ax=axs['A'])
trace = om.extract_traces(video_norm, (141, 100), size=10)
pacing_events = om.activation.find_activations(1 - trace, ax=axs['B'], fps=frequency)
plt.show()

print(f"Found {len(pacing_events)} activation events at frames: {pacing_events}")

Let's visualize the wave propagating across the ventricles for the first pacing beat:

In [ ]:
figure, axs = plt.subplots(1, 7, figsize=(10, 3))
om.show_image(video[0], ax=axs[0], title='original')
for i in range(0, 6):
    time = i*2 * (1000/frequency)  # convert frames to ms
    om.show_image(video_norm[pacing_events[0] + i*2], title=f"{time:.1f} ms", ax=axs[i+1])
plt.show()

## Computing Local Activation Times

Let's plot some of the optical traces (manually selected so that they show locations which become subsequently activated):

In [ ]:
# Positions selected with the GUI:
# positions = om.select_positions(video[0])
positions =  [(134, 101), (14, 93), (94, 99), (53, 97)]

fig, axs = plt.subplots(1, 2, figsize=(10,5))
om.trace.show_positions(positions, video[0], ax=axs[0])
traces = om.extract_traces(video_norm,
                           positions,
                           size=10,
                           fps=frequency,
                           ax=axs[1])
axs[1].axhline(y=0.5, color='r', linestyle='dashed', label='threshold')
axs[1].text(0.03, 0.52, 'threshold', color='r')
plt.xlim(0, 0.12)
plt.show()

The function {func}`find_activations` determines the time point at which the signal crosses the threshold (here 0.5). We will call this the local activation time (LAT). LATs can also be computed based on maximum derivative $\frac{dV}{dt}$, but this is not implemented in ``optimap`` yet.

```{note}
The function{func}`find_activations` will return the **closest** frame index to the threshold crossing. This means that if the threshold is crossed in between two frames, the function will return the index of the frame that is closest to the threshold crossing.
```

In [ ]:
threshold = 0.5
activations = om.activation.find_activations(traces, threshold=threshold, falling_edge=True, show=False)

fig, ax = plt.subplots()
colors = ['blue', 'orange', 'green', 'red']
xlim = (130, 146)
om.show_traces(traces, ax=ax, colors=colors, linestyle='solid', marker='.')
for i in range(len(activations)):
    for activation in activations[i]:
        if 130 <= activation <= 146:
            ax.axvline(activation, linestyle='--', color=colors[i], alpha=0.6)
            ax.text(activation, 0.45, f'LAT: {activation:.1f}', 
                    rotation=90, va='top', ha='right', color=colors[i], fontsize=10)
ax.axhline(y=threshold, color='r', linestyle='dashed')
ax.text(143.5, threshold + 0.02, 'Threshold', color='r')
ax.set_xlim(130, 146)
ax.grid()
fig.suptitle("Detected Local Activation Times (LATs)")
plt.show()

## Computing Activation Maps

We can now compute an activation map by identifying the local activation times in each pixel that correspond to when the action potential wave front passes through that pixel.

``optimap``'s {func}`compute_activation_map` function automatically computes a two-dimensional activation map which shows the local activation times in every pixel:

In [ ]:
t = pacing_events[2]
activation_map = om.compute_activation_map(
    video_norm[t - 2:t + 16],
    falling_edge=True,
    fps=frequency,
    vmax=15,
    show_contours=True
)

Note that we used the argument `falling_edge=True` due to the negative polarity of the signal ($- \Delta F / F$). If me had manually inverted the video beforehand or with calcium imaging data this would not be necessary.

Because our mask was automatically generated, we have a lot of pixels which are not part of the ventricles. Let's improve the mask manually to create better activation maps.

In [ ]:
# Refine mask manually with the GUI:
#
# mask = om.interactive_mask(image=video[0], initial_mask=mask)
# om.save_mask('mouse_41_120ms_control_iDS_mask.png', mask)

# Loading the mask from the file for demo purposes
mask_filename = om.download_example_data('mouse_41_120ms_control_iDS_mask.png')
mask = om.load_mask(mask_filename)
video_norm[:, mask] = np.nan

Now let's run it again. In the activation map below which show contour lines, which are a powerful visualization tool and help highlight the wavefront propagation. Contour lines are not shown by default, but can be added by setting `show_contours=True` in the {func}`compute_activation_map` or {func}`show_activation_map` functions. Here we also define custom contour levels of our choice:

In [ ]:
activation_map = om.compute_activation_map(
    video_norm[t - 3:t + 17],
    falling_edge=True,
    fps=frequency,
    show_contours=True,
    contour_levels=[3, 6, 9, 12],
    vmax=13,
)

The activation map is a 2D array with the LAT value in **frames** for each pixel. Both {func}`compute_activation_map` and {func}`find_activations` return results in terms of frames, the plotting functions {func}`activation.show_activations` and {func}`show_activation_map` convert it to milliseconds using the `fps` frame rate parameter.

If `normalize_time` is set to `True` (default) the minimum LAT is subtracted from all LAT values, so that the first activation time is at frame 0.

In [ ]:
om.print_properties(activation_map)

In this example the latest activation (LAT) at frame 16.

{func}`compute_activation_map` uses {func}`find_activation` to calculate the local activation time (LAT) for each pixel. By default the _closest_ time point to the threshold crossing is used, see left panel below.

When `interpolation=True` is set in either function, the LAT is calculated as the fractional time between the two frames that cross the threshold using linear interpolation (right panel):

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
ax=axs[0]
colors = ['blue', 'orange', 'green', 'red']
xlim = (130, 146)
for ax, interpolate in zip(axs, [False, True]):
    om.show_traces(traces, ax=ax, colors=colors, linestyle='solid', marker='.')
    activations = om.activation.find_activations(1 - traces, interpolate=interpolate, show=False)
    for i in range(len(activations)):
        for activation in activations[i]:
            if xlim[0] <= activation <= xlim[1]:
                ax.axvline(activation, linestyle='--', color=colors[i], alpha=0.6)
                ax.text(activation, 0.45, f'LAT: {activation:.1f}', 
                        rotation=90, va='top', ha='right', color=colors[i], fontsize=10)
    ax.axhline(y=0.5, color='r', linestyle='dashed')
    ax.text(xlim[1] - 3, 0.51, 'threshold', color='r')
    ax.set_xlim(*xlim)
    ax.grid()
    ax.set_title(f"interpolate=True" if interpolate else "interpolate=False (default)")
fig.suptitle("Non-interpolated vs interpolated LATs")
plt.show()

Here is the comparison of the activation maps with and without LAT interpolation:

In [ ]:
activation_map_smooth = om.compute_activation_map(
    video_norm[t-3:t+17],
    falling_edge=True,
    interpolate=True,
    show=False
)

fig, axs = plt.subplots(1, 3, figsize=(15, 6))
om.show_activation_map(activation_map,
                       ax=axs[0],
                       show_colorbar=False,
                       title='interpolate=False',
                       vmax=15)
om.show_activation_map(activation_map_smooth, 
                       ax=axs[1],
                       show_colorbar=False,
                       title='interpolate=True',
                       vmax=15)
om.show_activation_map(activation_map_smooth,
                       ax=axs[2],
                       show_contours=True,
                       contour_levels=[3, 6, 9, 12, 15],
                       contour_fmt=' %.1f ms ',
                       vmax=15,
                       title='with contours')
plt.show()

See {func}`show_activation_map` for more options to customize the activation map visualization.

Let's compare activation maps across subsequent pacing beats:

In [ ]:
fig, axs = plt.subplots(4, 3, figsize=(10, 10))
axs = axs.flatten()
for i in range(0, 12):
    t = pacing_events[i]
    om.compute_activation_map(
        video_norm[t - 4:t + 20],
        title=f"Beat {i}",
        falling_edge=True,
        fps=frequency,
        ax=axs[i],
        show_colorbar=False,
        vmax=16
    )
plt.tight_layout()
plt.show()


Let's look at beat 7 more closely, and overlay the contour lines on top of the raw image. The contour levels are set to 2 ms intervals, but you can adjust them as needed.

```{note}
You can also combine contour lines with a raw image to visualize the propagation path over the heart tissue:

In [ ]:
t = pacing_events[7]
activation_map = om.compute_activation_map(video_norm[t - 4:t + 20], falling_edge=True, interpolate=True, show=False)

fig, ax = plt.subplots(figsize=(8, 6))
# Show the raw camera image
om.show_image(video[t], ax=ax)
# Add a black boundary line around the mask
mask_boundary = ax.contour(~mask, levels=[0], colors='black', linewidths=1.5, alpha=0.8)
# Show contours
om.show_activation_map(
    activation_map,
    ax=ax,
    fps=frequency,
    show_map=False,
    show_contours=True,
    contour_levels=range(2, 20, 2),
    contour_fontsize=10,
    contour_args={'linewidths': 1.5, 'alpha': 0.8, 'cmap': 'turbo', 'colors': None})
plt.tight_layout()
plt.show()

We have plotted the activation map using the `turbo` colormap, but you can choose any colormap you prefer using the `cmap` argument. `vmin` and `vmax` can be used to set the minimum and maximum values for the color scale.

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(8, 4))
om.show_activation_map(activation_map, cmap='jet', show_colorbar=True, title='cmap=jet', ax=axs[0], fps=frequency, colorbar_title="", vmax=18)
om.show_activation_map(activation_map, cmap='magma', show_colorbar=True, title='cmap=magma', ax=axs[1], fps=frequency, colorbar_title="", vmax=18)
om.show_activation_map(activation_map, cmap='twilight_shifted', show_colorbar=True, fps=frequency, title='cmap=twilight_shifted', ax=axs[2], colorbar_title="", vmax=18)
plt.suptitle('Activation maps with different colormaps')
plt.show()

Using {func}`matplotlib.pyplot.quiver` we can visualize the propagation direction of the wavefront. The `quiver` function creates a 2D field of arrows.

In [ ]:
# Smooth activation map and calculate gradient of the activation map
activation_map_smooth2 = om.image.smooth_gaussian(activation_map_smooth, sigma=2)
dy, dx = np.gradient(activation_map_smooth2)

# Filter out 0.01% of the largest gradient values
gradient_magnitude = np.sqrt(dx**2 + dy**2)
threshold = np.nanpercentile(gradient_magnitude, 99.9)
dx[gradient_magnitude > threshold] = np.nan
dy[gradient_magnitude > threshold] = np.nan

step = 8  # adjust step size for arrow density
shape = activation_map_smooth2.shape
y_indices, x_indices = np.mgrid[:shape[0], :shape[1]]

fig, axs = plt.subplots(1, 2, figsize=(8, 4.5))
ax = axs[0]

om.show_activation_map(activation_map_smooth2, ax=ax, fps=frequency, vmax=15)
ax.quiver(x_indices[::step, ::step],
          y_indices[::step, ::step],
          dx[::step, ::step],
          dy[::step, ::step],
          color='black',
          pivot='mid',
          angles='xy',
          scale=4)
# Add a black boundary line around the mask
mask_boundary = ax.contour(~mask, levels=[0], colors='black', linewidths=1.5, alpha=0.8)

ax = axs[1]
step = 12
om.show_image(video[0], ax=ax)
ax.quiver(x_indices[::step, ::step],
          y_indices[::step, ::step],
          dx[::step, ::step],
          dy[::step, ::step],
          activation_map_smooth2[::step, ::step] / 15,
          width=0.005,
          cmap='turbo',
          pivot='mid',
          angles='xy',
          scale=3)

plt.tight_layout()
plt.show()

Similarly {func}`matplotlib.pyplot.streamplot` can also be used to visualize the propagation direction of the wavefront. The `streamplot` function creates a 2D field of streamlines, which are lines that follow the flow of the vector field.

In [ ]:
# Smooth the activation map and calculate the gradient
fig, axs = plt.subplots(1, 2, figsize=(8, 4.5))
ax = axs[0]
om.show_activation_map(activation_map_smooth2, ax=ax, fps=frequency, vmax=15)
ax.streamplot(x_indices, y_indices, dx, dy, color='white', linewidth=1)

ax = axs[1]
om.show_image(video[0], ax=ax)
ax.streamplot(x_indices, y_indices, dx, dy, color=activation_map_smooth2, cmap='turbo', linewidth=1, arrowsize=1.5)

fig.tight_layout()
fig.suptitle("Action Potential Wave Propagation Direction using streamplot")
plt.show()

## Computing Activation Maps from Frame-Wise Difference Optical Maps

```{warning}
This rest of this tutorial is currently work in progress. We will add more information soon.
```

In [Tutorial 2](signal_extraction.ipynb), we introduced the frame-wise difference method to emphasize sudden temporal changes in a video. Sudden temporal changes are caused by upstrokes of the action potential or calcium transients and the frame-wise difference filter is therefore ideally suited to visualize wavefronts as they propagate across the tissue.

In [ ]:
video_diff = om.video.temporal_difference(video_filtered, 5)
video_diff[:, mask] = np.nan
video_diff_norm = om.video.normalize_pixelwise(video_diff)

The frame-wise difference approach enhances action potential upstroke, see the following video with temporal difference in the middle and our previous pixel-wise normalized video on the right:

In [ ]:
om.show_videos([video, video_diff_norm, video_norm],
               titles=["original", "frame-wise diff", "pixelwise normalized"],
               interval=100)

In [ ]:
render(lambda: om.show_videos([video, video_diff_norm, video_norm],
               titles=["original", "frame-wise diff", "pixelwise normalized"],
               interval=250))

Let's visualize the wavefront as an overlay over the raw (motion-stabilized) video. We will need to further post-process the data as follows: 

In [ ]:
video_diff[video_diff > 0] = 0
video_diff_norm = om.video.normalize_pixelwise(-video_diff)

The action potential upstroke overlaid onto the raw video:

In [ ]:
om.video.show_video_overlay(video,
                            overlay=video_diff_norm,
                            vmin_overlay=-1,
                            vmax_overlay=1)

In [ ]:
render(lambda: om.video.show_video_overlay(video, video_diff_norm, vmin_overlay=-1, vmax_overlay=1, interval=200))